In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from scipy.io import loadmat
from scipy.sparse import dok_matrix
from scipy.sparse.csgraph import connected_components
import numpy  as np
import sys
import solution
import helper

sys.path.append('../')
import commons
from commons import factor
from commons.factor import Factor
import drandom

## Block Log Distribution:

In [2]:
# This cell prepares the arguments.
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)

inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']
V, G, F, A = [inp['t6a1']-1], inp['t6a2'], inp['t6a3'], inp['t6a4']-1

G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
F = np.array([Factor.from_matlab(f) for f in F])

In [3]:
out['t6'] == solution.block_log_distribution(V, G, F, A)

array([ True,  True])

## Gibbs trans

In [4]:
drandom.seed(1)

In [5]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)

inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']

As, Gs, Fs = inp['t7a1'], inp['t7a2'], inp['t7a3']

for i in range(10):
    A, G, F = As[i], Gs[i], Fs[i]
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A = A-1
    real_out = out['t7'][i]-1
    my_out = solution.gibbs_trans(A, G, F)
    print(np.all(real_out == my_out))
    

True
True
True
True
True
True
True
True
True
True


## MCMC Inference Part I

In [6]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)
inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']
drandom.seed(1)
for i in range(2):
    G, F, E, trans_name, mix_time, num_samples, sampling_interval, A0 = (inp[f't8a{j}'][i] for j in range(1, 9))
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A0 = A0-1
    
    if i == 1:
        trans_name = 'MHGibbs'
    M_true = [Factor.from_mat_struct(f) for f in out['t8o1'][i]]
    all_samples_true = out['t8o2'][i] - 1
    
    M, all_samples = solution.mcmc_inference(G, F, E, trans_name, mix_time, num_samples, sampling_interval, A0)
    print(M_true == M, np.all(all_samples == all_samples_true))

Running the markov chain.:   8%|▊         | 40/500 [00:00<00:01, 389.80it/s]

True True


Running the markov chain.: 100%|██████████| 500/500 [00:01<00:00, 391.18it/s]

True True


## MH Uniform Trans

In [7]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)
inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']

As, Gs, Fs = inp['t9a1'],inp['t9a2'], inp['t9a3']
drandom.seed(1)
for i in range(10):
    A, G, F = As[i], Gs[i], Fs[i]
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A = A-1
    real_out = out['t9'][i]-1
    my_out = solution.mh_uniform_trans(A, G, F)
    print(np.all(real_out == my_out))

True
True
True
True
True
True
True
True
True
True


## MHSW Part 1

In [8]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)
inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']

As, Gs, Fs = inp['t10a1'],inp['t10a2'], inp['t10a3']
drandom.seed(1)
for i in range(10):
    
    A, G, F = As[i], Gs[i], Fs[i]
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    G['q_list'][:, 0] -= 1
    G['q_list'][:, 1] -= 1
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A = A-1
    real_out = out['t10'][i]-1
    my_out = solution.mhsw_trans(A, G, F, variant=1)
    print(np.all(real_out == my_out))

True
True
True
True
True
True
True
True
True
True


## MHSW Part 2

In [9]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)
inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']

As, Gs, Fs = inp['t11a1'],inp['t11a2'], inp['t11a3']
drandom.seed(1)
for i in range(10):
    
    A, G, F = As[i], Gs[i], Fs[i]
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    G['q_list'][:, 0] -= 1
    G['q_list'][:, 1] -= 1
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A = A-1
    real_out = out['t11'][i]-1
    my_out = solution.mhsw_trans(A, G, F, variant=2)
    print(np.all(real_out == my_out))

True
True
True
True
True
True
True
True
True
True


## MCMC Inference Part II

In [10]:
sample = loadmat('./data/exampleIOPA5.mat', simplify_cells=True)
inp = sample['exampleINPUT']
out = sample['exampleOUTPUT']
seeds = [1, 26288942]
for i in range(2):
    drandom.seed(seeds[i])
    G, F, E, trans_name, mix_time, num_samples, sampling_interval, A0 = (inp[f't12a{j}'][i] for j in range(1, 9))
    G['adj_list'] = commons.adj_matrix_to_adj_list(G['edges'])
    G['var2factors'] = np.array([x-1 for x in G['var2factors']], dtype='object')
    F = np.array([Factor.from_mat_struct(f) for f in F])
    A0 = A0-1
    
    M_true = [Factor.from_mat_struct(f) for f in out['t12o1'][i]]
    all_samples_true = out['t12o2'][i] - 1
    
    M, all_samples = solution.mcmc_inference(G, F, E, trans_name, mix_time, num_samples, sampling_interval, A0)
    print(M_true == M, np.all(all_samples == all_samples_true))

Running the markov chain.:  12%|█▏        | 60/500 [00:00<00:00, 591.56it/s]

True True


Running the markov chain.: 100%|██████████| 500/500 [00:00<00:00, 565.11it/s]

True True
